## Climatic data



In [34]:
import ee
import geemap

# Authenticate to Earth Engine
try:
  ee.Initialize()
except Exception as e:
  ee.Authenticate()
  ee.Initialize()

# data_folder = 'projects/amazon-forest-regrowth/assets'
roi = ee.FeatureCollection("projects/amazon-forest-regrowth/assets/raw/biomes_br").geometry().dissolve()

# months = ee.List.sequence(1, 12)
# years = ee.List.sequence(1985, 2020)

first_year = 1985
last_year = 2020
# 1986 - 2019, years included in analysis
years = range((first_year + 1), last_year)

ecoregions = (ee.FeatureCollection("RESOLVE/ECOREGIONS/2017").filterBounds(roi)
                .map(lambda feature: feature.intersection(roi)))

# Terraclim
  https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE
Bring temperature and precipitation and calculate seasonality

In [42]:
terraclim = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE') \
              .filterDate('1985-01-01', '2019-12-31') \
              .map(lambda image : image.clip(roi)) \
              .map(lambda image: image.reduceResolution(ee.Reducer.median(), bestEffort=True, maxPixels=1024) \
                                       .reproject(crs='EPSG:4326', scale=10000))

# scaling to 0.1
maxtemp = terraclim.select('tmmx').map(lambda image: image.multiply(0.1))
mintemp = terraclim.select('tmmn').map(lambda image: image.multiply(0.1))
radiation = terraclim.select('srad').map(lambda image: image.multiply(0.1))
prec = terraclim.select('pr')

def export_image(img, name):
    # Create the export task
    task = ee.batch.Export.image.toAsset(
        image = img.slice(1),
        description = f'{name}',
        assetId = f'projects/amazon-forest-regrowth/assets/{name}',
        scale = 10000,
        crs = 'EPSG:4326',
        maxPixels = 4e12
    )
    # Start the export task
    task.start()

## Precipitation

For each regrowing forest pixel, we only consider rainfall AFTER the regrowth moment.
- Make a raster with mean yearly precipitation, temperature, and precipitation
- select only the values for years after regrowth. For years before regrowth, change the values to zero.

## Seasonality index

Walsh and Lawler 1981

In [47]:
yearly_SI = ee.Image()
yearly_mean_prec = ee.Image()
yearly_mean_temp = ee.Image()

for year in years:    # get mean monthly precipitation for the year
    yr_prec = prec.filter(ee.Filter.calendarRange(year, year, 'year'))
    mean_prec = yr_prec.reduce(ee.Reducer.mean())
    yearly_mean_prec = yearly_mean_prec.addBands(mean_prec.toFloat().rename(f"prec_{year}"))
    # Calculate absolute deviations of monthly precipitation from mean
    deviations = yr_prec.map(lambda month:month.subtract(mean_prec).abs())
    # Calculate sum of absolute deviations for each month
    sum_deviations = deviations.reduce(ee.Reducer.sum())
    # Calculate total annual precipitation
    total_annual_prec = yr_prec.reduce(ee.Reducer.sum())
    # Calculate Seasonality Index (SI)
    yearly_SI = yearly_SI.addBands(sum_deviations.divide(total_annual_prec).toFloat().rename(f"si_{year}"))
    # Filter both maxtemp and mintemp ImageCollections by year at once
    yearly_maxtemp = maxtemp.filter(ee.Filter.calendarRange(year, year, 'year')).reduce(ee.Reducer.mean())
    yearly_mintemp = mintemp.filter(ee.Filter.calendarRange(year, year, 'year')).reduce(ee.Reducer.mean())
    # Calculate the mean temperature directly
    yearly_mean_temp = yearly_mean_temp.addBands(yearly_maxtemp.add(yearly_mintemp).divide(2) \
                                        .toFloat().rename(f"mean_temp_{year}"))

export_image(yearly_mean_prec, "yearly_mean_prec")
export_image(yearly_SI, "yearly_SI")
export_image(yearly_mean_temp, "yearly_mean_temp")

## Yearly CWD

Calculated as in Malhi et al 2009 and Aragão et al 2007, considering ET fixed as 100mm/month.

[equations](https://imgur.com/o4lVmM7)

In [46]:
"""
Calculate monthly cumulative water deficit
"""
prec_list = prec.toList(prec.size())
months_list = range(2, prec.size().getInfo()+1)
cwd_monthly = ee.Image()

for month in prec_list:
    current_image = prec_list.get(month)
    previous_val = cwd_monthly.select(-1)
    result_image = previous_val.add(current_image).subtract(100)
    result_image = result_image.where(result_image.gt(0), 0) \
                                   .set('system:time_start', current_image.get('system:time_start'))
    return ee.Dictionary({
        'result_image': ee.Image(result_image).toFloat(),
        'cwd_list': ee.List(previous_dict.get('cwd_list')).add(result_image)})

# # Use iterate to apply the function to each image
# cwd_dict = ee.Dictionary(prec.iterate(cwd_monthly, ee.Dictionary({'result_image': ee.Image(0).toFloat(),
#                                                                'cwd_list': ee.List([])})))
# cwd_list = ee.List(cwd_dict.get('cwd_list'))
# # cwd_list = cwd_list.map(lambda image: image.toFloat())

# def mcwd_yearly(year):
#     # Calculate the start and end indices for the current year
#     start = ee.Number(year).subtract(1985).multiply(12)
#     end = start.add(12)
#     # Slice the list to get the images for the current year
#     yr = ee.List(cwd_list).slice(start, end)
#     cwd_a = ee.ImageCollection.fromImages(yr)
#     mcwd_a = cwd_a.reduce(ee.Reducer.min())
#     return mcwd_a.set('year', year)

# # Map the function over the range
# yearly_mcwd_img = ee.ImageCollection.fromImages(years.map(mcwd_yearly)).toBands()

# yearlist = range(1985, 2020) # Generate a list of years from 1985 to 2019
# new_band_names = ['mcwd_{}'.format(year) for year in yearlist] # Append 'si_' to each year
# yearly_mcwd_img = yearly_mcwd_img.rename(new_band_names)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
